In [1]:
from pathlib import Path
import re
import requests
import time
import datetime

import pandas as pd
from requests_html import HTML
from selenium import webdriver
from selenium.webdriver.chrome.options import Options
options = Options()
options.add_argument("--headless")
driver = webdriver.Chrome(executable_path='/Users/ruby/Downloads/Chrome Driver/chromedriver', options=options)

/var/folders/3q/3sbzpm556tn_jsn0vdddr77c0000gp/T/ipykernel_52895/2416605280.py:13: DeprecationWarning: executable_path has been deprecated, please pass in a Service object
  driver = webdriver.Chrome(executable_path='/Users/ruby/Downloads/Chrome Driver/chromedriver', options=options)


In [2]:
BASE_DIR = Path.cwd()
DATA_DIR = BASE_DIR / "data" # os.path.join(BASE_DIR, 'data')
if not DATA_DIR.exists(): # os.path.exists(DATA_DIR)
    DATA_DIR.mkdir(exist_ok=True) # os.makedirs(DATA_DIR, exist_ok=True)
    
product_category_links_output = DATA_DIR / "category-products.csv"
product_output = DATA_DIR / "products.csv"

In [3]:
#options = Options()
#options.add_argument("--headless")

#driver = webdriver.Chrome(options=options)

In [4]:
categories = [
    {"name": "toys-and-games", "url": "https://www.amazon.com/Best-Sellers-Toys-Games/zgbs/toys-and-games/"},
    {"name": "electronics", "url": "https://www.amazon.com/Best-Sellers-Electronics/zgbs/electronics/"},
    {"name": "fashion", "url": "https://www.amazon.com/Best-Sellers/zgbs/fashion/"}
]

In [5]:
categories

[{'name': 'toys-and-games',
  'url': 'https://www.amazon.com/Best-Sellers-Toys-Games/zgbs/toys-and-games/'},
 {'name': 'electronics',
  'url': 'https://www.amazon.com/Best-Sellers-Electronics/zgbs/electronics/'},
 {'name': 'fashion',
  'url': 'https://www.amazon.com/Best-Sellers/zgbs/fashion/'}]

In [6]:
#lookng through the above pattern of website scraped, we see a general pattern, listed below
regex_options = [
    r"https://www.amazon.com/gp/product/(?P<product_id>[\w-]+)/",
    r"https://www.amazon.com/dp/(?P<product_id>[\w-]+)/",
    r"https://www.amazon.com/(?P<slug>[\w-]+)/dp/(?P<product_id>[\w-]+)/",
]

def extract_product_id_from_url(url):
    product_id = None
    for regex_str in regex_options:
        regex = re.compile(regex_str)
        match = regex.match(url)
        if match != None:
            try:
                product_id = match['product_id']
            except:
                pass
    return product_id

In [7]:
#page_links = [f"https://www.amazon.com{x}" for x in html_obj.links if x.startswith("/")]
# page_links = [x for x in page_links if extract_product_id_from_url(x) != None]
#for url in page_links:
 #   product_id = extract_product_id_from_url(x)

    #to perform scrape
def clean_page_links(page_links=[], category=None):
    final_page_links = []
    for url in page_links:
        product_id = extract_product_id_from_url(url)
        if product_id != None:
            final_page_links.append({"url": url, "product_id": product_id, "category": category})
    return final_page_links

In [8]:
def scrace_category_product_links(categories=[]):
    all_product_links = []
    for category in categories:
        time.sleep(1.5)
        url = category.get("url")
        driver.get(url)
        body_el = driver.find_element_by_css_selector("body")
        html_str = body_el.get_attribute("innerHTML")
        html_obj = HTML(html=html_str)
        page_links = [f"https://www.amazon.com{x}" for x in html_obj.links if x.startswith("/")]
        cleaned_links = clean_page_links(page_links=page_links, category=category)
        all_product_links += cleaned_links
    return all_product_links

In [9]:
#print(all_product_links)

In [10]:
def extract_categories_and_save(categories=[]):
    all_product_links = scrace_category_product_links(categories)
    category_df = pd.DataFrame(all_product_links)
    category_df.to_csv(product_category_links_output, index=False)

In [11]:
extract_categories_and_save(categories=categories)

/var/folders/3q/3sbzpm556tn_jsn0vdddr77c0000gp/T/ipykernel_52895/2215756588.py:7: DeprecationWarning: find_element_by_css_selector is deprecated. Please use find_element(by=By.CSS_SELECTOR, value=css_selector) instead
  body_el = driver.find_element_by_css_selector("body")


In [12]:
def scrape_product_page(url, title_lookup = "#productTitle", price_lookup = "#priceblock_ourprice"):
    driver.get(url)
    time.sleep(1.5)
    body_el = driver.find_element_by_css_selector("body")
    html_str = body_el.get_attribute("innerHTML")
    html_obj = HTML(html=html_str)
    product_title = html_obj.find(title_lookup, first=True).text
    product_price = html_obj.find(price_lookup, first=True).text
    return product_title, product_price

In [13]:
def perform_scrape(cleaned_items=[]):
    data_extracted = []
    for obj in cleaned_items:
        link = obj['url']
        product_id = obj['product_id']
        title, price = (None, None)
        try:
            title, price = scrape_product_page(link)
        except:
            pass
        if title != None and price != None:
            print(link, title, price)
        product_data = {
            "url": link,
            "product_id": product_id,
            "title": title,
            "price": price
        }
        data_extracted.append(product_data)
    return data_extracted

In [14]:
#extracted_data = perform_scrape(cleaned_items=cleaned_items)

In [15]:
#print(extracted_data)

In [25]:
def row_scrape_event(row, *args, **kwargs):
    link = row['url']
    scraped = 0
    try:
        scraped = row['scraped']
    except:
        pass
    # print(link)
    if scraped == 1 or scraped == "1":
        print("skipped")
        return row
    product_id = row['product_id']
    title, price = (None, None)
    try:
        title, price = scrape_product_page(link)
    except:
        pass
    row['title'] = title
    row['price'] = price
    row['scraped'] = 1
    row['timestamp'] = datetime.datetime.now().timestamp()
    print(link, title, price)
    return row

In [30]:
print(product_id)

product_id


In [17]:
df = pd.read_csv(product_category_links_output)
df.head()

,url,product_id,category
0,https://www.amazon.com/Glow-Sticks-Bulk-Party-...,B07ZV31J31,"{'name': 'toys-and-games', 'url': 'https://www..."
1,https://www.amazon.com/Bunch-Balloons-Zuru-Sel...,B07FL35M4Z,"{'name': 'toys-and-games', 'url': 'https://www..."
2,https://www.amazon.com/Aqua-Monterey-Multi-Pur...,B073WMYP6M,"{'name': 'toys-and-games', 'url': 'https://www..."
3,https://www.amazon.com/Barbie-Reveal-Costume-S...,B09KTFNNZ1,"{'name': 'toys-and-games', 'url': 'https://www..."
4,https://www.amazon.com/Kinetic-Sand-Online-Exc...,B09BW36FW6,"{'name': 'toys-and-games', 'url': 'https://www..."


In [18]:
df.shape

(88, 3)

In [19]:
df_sub = df.copy() # df.head(n=10)

In [20]:
df_sub = df_sub.apply(row_scrape_event, axis=1)

/var/folders/3q/3sbzpm556tn_jsn0vdddr77c0000gp/T/ipykernel_52895/684386146.py:4: DeprecationWarning: find_element_by_css_selector is deprecated. Please use find_element(by=By.CSS_SELECTOR, value=css_selector) instead
  body_el = driver.find_element_by_css_selector("body")


https://www.amazon.com/Glow-Sticks-Bulk-Party-Supplies/dp/B07ZV31J31/ref=zg_bs_toys-and-games_9/134-8452567-0500404?pd_rd_i=B08B146216&psc=1 None None


/var/folders/3q/3sbzpm556tn_jsn0vdddr77c0000gp/T/ipykernel_52895/684386146.py:4: DeprecationWarning: find_element_by_css_selector is deprecated. Please use find_element(by=By.CSS_SELECTOR, value=css_selector) instead
  body_el = driver.find_element_by_css_selector("body")


https://www.amazon.com/Bunch-Balloons-Zuru-Self-Sealing-Water/dp/B07FL35M4Z/ref=zg_bs_toys-and-games_13/134-8452567-0500404?pd_rd_i=B09T9P47P9&psc=1 None None


/var/folders/3q/3sbzpm556tn_jsn0vdddr77c0000gp/T/ipykernel_52895/684386146.py:4: DeprecationWarning: find_element_by_css_selector is deprecated. Please use find_element(by=By.CSS_SELECTOR, value=css_selector) instead
  body_el = driver.find_element_by_css_selector("body")


...

https://www.amazon.com/Cushionaire-Womens-Feather-recovery-Comfort/dp/B09DMZ7JTV/ref=zg_bs_fashion_11/134-8452567-0500404?pd_rd_i=B094SS9VK9&psc=1 None None


/var/folders/3q/3sbzpm556tn_jsn0vdddr77c0000gp/T/ipykernel_52895/684386146.py:4: DeprecationWarning: find_element_by_css_selector is deprecated. Please use find_element(by=By.CSS_SELECTOR, value=css_selector) instead
  body_el = driver.find_element_by_css_selector("body")


https://www.amazon.com/Levis-Womens-Plus-Size-Original-Mood-Dark/dp/B096MYZQ9H/ref=zg_bs_fashion_24/134-8452567-0500404?pd_rd_i=B09RQTMLXP&psc=1 None None


/var/folders/3q/3sbzpm556tn_jsn0vdddr77c0000gp/T/ipykernel_52895/684386146.py:4: DeprecationWarning: find_element_by_css_selector is deprecated. Please use find_element(by=By.CSS_SELECTOR, value=css_selector) instead
  body_el = driver.find_element_by_css_selector("body")


https://www.amazon.com/Womens-Wedding-V-Neck-Lantern-Dresses/dp/B09L15H5VG/ref=zg_bs_fashion_29/134-8452567-0500404?pd_rd_i=B09DS5ZMZ2&psc=1 None None


In [32]:
df.to_csv(product_output, index=False)

In [33]:
products_df = pd.read_csv(product_output)

In [34]:
final_df = pd.concat([products_df, df_sub])
final_df.to_csv(product_output, index=False)

In [35]:
final_df.head()

,url,product_id,category,title,price,scraped,timestamp
0,https://www.amazon.com/Glow-Sticks-Bulk-Party-...,B07ZV31J31,"{'name': 'toys-and-games', 'url': 'https://www...",NaN,NaN,NaN,NaN
1,https://www.amazon.com/Bunch-Balloons-Zuru-Sel...,B07FL35M4Z,"{'name': 'toys-and-games', 'url': 'https://www...",NaN,NaN,NaN,NaN
2,https://www.amazon.com/Aqua-Monterey-Multi-Pur...,B073WMYP6M,"{'name': 'toys-and-games', 'url': 'https://www...",NaN,NaN,NaN,NaN
3,https://www.amazon.com/Barbie-Reveal-Costume-S...,B09KTFNNZ1,"{'name': 'toys-and-games', 'url': 'https://www...",NaN,NaN,NaN,NaN
4,https://www.amazon.com/Kinetic-Sand-Online-Exc...,B09BW36FW6,"{'name': 'toys-and-games', 'url': 'https://www...",NaN,NaN,NaN,NaN
